# TP 1 et 2 : Accès aux données avec index
# sujet


NOM: BOUZOURINE

Prénom: HICHEM



TP à rendre : **REDIGER des explications détaillées et argumentées** pour les solutions que vous proposez




Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'une fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [ ]:
import os
import shutil as sh
import numpy as np
import pandas as pd
import random

# from sortedcontainers import SortedDict
import sortedcontainers

from string import ascii_lowercase
import time

# le nom de la table
TABLE = "T"
print("le nom de la table est", TABLE)


# le nom du fichier qui contient les données de la table
def nom_fichier(table):
    return table + ".csv"

# Générer les données du TP

Création du fichier contenant un exemple de données.
Ce sont des données au format csv. On suppose que chaque ligne correspond à un tuple d'une table **T** ayant *n* attributs :

$$ T (a_0, a_1, a_2, ..., a_{n-1})$$

Le premier attribut $a_0$ est unique.

Les attributs $a_1$ à $a_{n-2}$ ne sont pas uniques : il y a en moyenne $2^k$ tuples par valeurs de $a_k$ soit 2 tuples par valeur de $a_1$ et 4 tuples par valeurs de $a_2$.

Les attributs sont des nombres entiers sauf le dernier qui est une chaine de caractères.


In [ ]:
# dure environ 20s pour 2M lignes
# dure environ 40s pour 5M lignes


def genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, table):
  # attribut_chaine_caracteres = "".join(choice(ascii_lowercase) for i in range(longueur_dernier_attribut))
  attribut_chaine_caracteres = ''.join('-' for i in range(longueur_dernier_attribut))
  # print("le dernier attribut de chaque tuple est la chaine de caracètes :", attribut_chaine_caracteres)

  # reproductibilité des données générées
  rng = np.random.default_rng(seed=1)

  data={}

  # le premier attribut est unique.
  nb_valeurs_distinctes = nb_lignes
  data['a0'] = 10 * rng.permutation(np.arange(nb_valeurs_distinctes))

  # les attributs suivants ont des domaines plus petits :
  for i in range(1, nb_attributs):
    # on divise le domaine par 2 à chaque itération
    nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
    data[f'a{i}'] = 10 * rng.integers(0, nb_valeurs_distinctes, nb_lignes)

  # on concatène "verticalement" les attributs dans un dataframe pour former des tuples sur lesquels on peut itérer.
  df = pd.DataFrame(data)
  # rmq: le dernier attribut est une chaine de caractères
  b = [str(e)[1:-1] + f",{attribut_chaine_caracteres}\n" for e in df.itertuples(index=False, name=None)]

  # on stocke les données dans un fichier
  fichier = nom_fichier(table)
  print("écriture des données dans le fichier", fichier)
  with open(fichier, "w") as f:
    # écriture groupée de tous les tuples
    f.write(''.join(b))

nb_lignes = 2 * 1000 * 1000
# nb_lignes = 5 * 1000 * 1000
nb_attributs = 7
longueur_dernier_attribut = 100

t1 = time.time()
genere_fichier(nb_lignes, nb_attributs, longueur_dernier_attribut, TABLE)
print(f"durée pour générer {nb_lignes} lignes :", round(time.time() - t1, 1), "s")

On affiche le début et la fin du fichier et son nombre de lignes ( = card(T))

In [ ]:
%%bash -s "{TABLE}.csv"
echo "debut de $1 :"
head -n 2 $1
echo
echo "fin de $1 : "
tail -n 2 $1
echo
echo "nombre de lignes:"
wc -l $1


In [ ]:
df = pd.read_csv(nom_fichier(TABLE))
df.head()

# Lecture séquentielle

On définit un *iterateur* pour lire séquentiellement chaque ligne de la table stockée entièrement dans un seul fichier.
Le mot python *yield* permet de définir un itérateur qui est retourné par la fonction.

Cet itérateur est invoqué pour lire la table et appliquer un filtre.



In [ ]:
def lecture_sequentielle(table):
  fichier = nom_fichier(table)
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      yield i, line

def filtrer_table(table, valeur_recherchee):
  for i, line in lecture_sequentielle(table):
      a = int(line.split(',')[0])
      if a == valeur_recherchee :
        print(f"ligne {i} :", line.strip())




In [ ]:
nb_valeurs_distinctes = nb_lignes
s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_table(TABLE, s)
print("durée :", round(time.time() - t1, 3), "s")

# Découper une table en pages

On organise les données en pages.
Pour faciliter le TP, chaque page est représentée par un "petit" fichier mais en réalité une page est un bloc d'un fichier.

Dans la suite du TP, on accédera toujours aux pages.
Le fichier créé initialement, contenant tous les tuples, ne sera plus utilisé.

In [ ]:
def page_dir_name(table):
  return table + "_pages"


In [ ]:

def decoupe_table_en_pages(table, nb_tuple_par_page):
  page_dir = page_dir_name(table)

  # vider le dossier qui contiendra les pages
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  # lire le fichier contenant tous les tuples
  p=0
  lines = []
  for i, line in lecture_sequentielle(table):
    lines.append(line)
    if (i+1) % nb_tuple_par_page == 0:

      # créer une page
      p += 1
      with open(page_dir + f"/page{p}.csv", "w") as fp:
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(page_dir + f"/page{p}.csv", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p)


print("les pages sont stockées dans le dossier", page_dir_name(TABLE) )

decoupe_table_en_pages(TABLE, nb_tuple_par_page=1000)

Afficher (pour quelques pages) le nombre de tuples contenus dans une page

In [ ]:
df = pd.read_csv('./T_pages/page1.csv')
df.head()

In [ ]:
rows,columns =df.shape  
print(f'contient : {rows} ligne et {columns} colonne')

une solution en bash :

In [ ]:
%%bash -s "$TABLE"
wc -l $1_pages/* | head -n 3

une autre solution en python :

In [ ]:
page_dir = page_dir_name(TABLE)
l = os.listdir(page_dir)
random.seed(1)
for i in range(3):
  une_page = random.choice(l)
  with open(page_dir + f"/{une_page}", 'r') as fp:
    lines = len(fp.readlines())
    print(f"la page {une_page} contient {lines} lignes")

# Lecture séquentielle d'une table découpée en pages

In [ ]:
def lecture_sequentielle_par_page(table):
  page_dir = page_dir_name(table)
  nb_pages = len(os.listdir(page_dir))
  for p in range(1, nb_pages+1) :
    with open(page_dir + f"/page{p}.csv", "r") as f:
      for i, line in enumerate(f):
        tuple_courant = line.strip().split(',')
        yield p, i, tuple_courant

def filtrer_table_par_pages(table, valeur_recherchee):
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    attribut0 = int(tuple_courant[0])
    if attribut0 == valeur_recherchee :
      print(f"page {page}, ligne {position} :", tuple_courant)


In [ ]:

search = np.random.randint(nb_lignes)
print("valeur recherchée :", search)

t1 = time.time()
filtrer_table_par_pages(TABLE, search)
print("durée :", round(time.time() - t1, 2), "s")

In [ ]:
def lecture_tuple(table, num_page, position):
  page_dir = page_dir_name(table)
  with open(page_dir + f"/page{num_page}.csv", "r") as f:
    lines = f.readlines()
    return lines[position].strip()

In [ ]:
t1 = time.time()
print(lecture_tuple(TABLE, 123, 456))
print("done in", round((time.time() - t1)*1000, 1), "ms")

# Exercice 1 : Créer un index

## Créer un index unique pour l'attribut $a_0$

On sait que $a_0$ est unique.
Une entrée de l'index associe une *clé* à une *valeur* :
*   La *clé* est la valeur du premier attribut.
*   La *valeur* est un **rowid** formé des informations (page, position)



In [ ]:
def creation_index_unique(table)->sortedcontainers.SortedDict:
  index = {}
  page_dir = page_dir_name(table)
  nb_pages = len(os.listdir(page_dir))
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    a0 = int(tuple_courant[0])
    index[a0]= {'page':page,'position':position}     



  return sortedcontainers.SortedDict(index)



In [ ]:
t1 = time.time()
INDEX_UNIQUE_a0 = creation_index_unique(TABLE)
print("durée ", round(time.time() - t1, 3), "s")

In [ ]:
# vérifier l'index
s = 10 * np.random.randint(nb_lignes)
print(s, INDEX_UNIQUE_a0[s])
# print(s, INDEX_UNIQUE_a0[19512870])

## Créer un index non unique pour l'attribut $a_i$

On donne un nom de table et le numéro $i$ de l'attribut $a_i$ de la table.

In [ ]:
def creation_index(table, numero_attribut_i)->sortedcontainers.SortedDict:
  index = {}
  page_dir = page_dir_name(table)
  nb_pages = len(os.listdir(page_dir))
  for page, position, tuple_courant in lecture_sequentielle_par_page(table):
    
    ai = int(tuple_courant[numero_attribut_i]) # ai contient le numéro, par exemple 7
    # si il dans l'index le rajouter dans la liste : e.g : {7:[{position:2,page:1}]}
    # sinon crée une nouvelle liste avec cette élément 
    # Nom : {ai : [{page:0,position:2},{page:2, position : 1000}]}
    ai_position = {'page':page,'position':position}
    if(index.get(ai) is None) : 
      index[ai] = [ai_position]    
    else : 
      index[ai].append(ai_position)
      
  return sortedcontainers.SortedDict(index)




In [ ]:
t1 = time.time()
INDEX_a2 = creation_index(TABLE, 2)
print("duree de création de l'index pour l'attribut a2:", round(time.time() - t1, 3), "s")

In [ ]:
INDEX_a2.get(1235270)

In [ ]:
# vérifier l'index
s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)
for r in INDEX_a2[s]:
  print(r)

In [ ]:
INDEX_a2.get(1235270)

# Exerccie 2 : Accès par index

## Accès ciblé

On veut retrouver les tuples telq qu'un attribut indexé a une valeur donnée.

###Index unique scan.
On recherche un unique tuple dont l'attribut indexé a une valeur donnée (car l'attribut est unique)

In [ ]:
def acces_par_index_unique(index_unique:object, table:str, valeur_recherchee:int):
    position_page = index_unique[valeur_recherchee]
    if position_page is None : 
        return None 
    # cas indice existe 
    page,position = position_page['page'],position_page['position']
        

    return lecture_tuple(table, page, position)




In [ ]:
s = 10 * np.random.randint(nb_lignes)
print("valeur recherchée :", s)

t1 = time.time()
tuple = acces_par_index_unique(INDEX_UNIQUE_a0, TABLE, s)
print("resulat:", tuple)
print("done in", round((time.time() - t1)*1000, 2), "ms")

###Index scan
Accès pour rechercher les tuples dont l'attribut indexé a une valeur donnée. On suppose que l'attribut n'est pas unique et que plusieurs tuples sont retrouvés

In [ ]:
def acces_par_index(index, table, valeur_recherchee):
    rows = index[valeur_recherchee]
    if rows is None : 
        return [] 
    # cas indice existe 
    tuples = []
    for row in rows : 
        page,position = row['page'],row['position']
        tuple = lecture_tuple(table, page, position)
        tuples.append(tuple)
        # yield p, i, tuple_courant
    
    return tuples

In [ ]:
INDEX_a2[1385910]


In [ ]:
# s = 1385910
s = 10* np.random.randint(nb_lignes/4)
print("valeur recherchée :", s)

t1 = time.time()
for t in acces_par_index(INDEX_a2, TABLE, s):
  print(t)
print("done in", round((time.time() - t1)*1000, 2), "ms")

## Accès par intervalle
Index range scan



### Accès par intervalle sur un attribut unique
Accès pour rechercher les tuples dont l'attribut indexé est unique et a une valeur comprise dans un intervalle donné.
Indications, votre solution doit prendre en compte les exigences suivantes :
*  Les valeurs recherchées ne sont pas connues à l'avance. On sait seulement qu'elles sont incluses dans un intervalle. Ne pas supposer qu'on recherche des entiers consécutifs.
*  Les bornes de l'intervalle ne sont pas parmi les valeurs existantes de l'attribut. Par exemple, on peut rechercher les valeurs de $a_0$ comprises dans l'intervalle  [23 , 45].



In [ ]:
# Exemple pour retrouver la première entrée de l'intervalle [23,45]

indice = INDEX_UNIQUE_a0.bisect_left(23)
print("l'indice de la première clé à retrouver est :", indice)
cle = INDEX_UNIQUE_a0.keys()[indice]
print("la première clé retrouvée est:", cle)
print("la valeur à retrouver est :",  INDEX_UNIQUE_a0[cle])

In [ ]:
def acces_intervalle_par_index_unique(index_unique:object, table, borne_inf, borne_sup):

    indice_inf = index_unique.bisect_left(borne_inf)
    indice_sup = index_unique.bisect_right(borne_sup)
    tuples = []
    while indice_inf<= indice_sup : 
        cle = index_unique.keys()[indice_inf]
        tuple = acces_par_index_unique(index_unique,table,cle)
        indice_inf +=1
        tuples.append(tuple) # utiliser le yield
    return tuples

In [ ]:
INDEX_UNIQUE_a0[2046210]

In [ ]:
s = 10 * np.random.randint(nb_valeurs_distinctes/4)
print("valeur recherchée :", s)

t1 = time.time()
tuples_indices = acces_intervalle_par_index_unique(INDEX_UNIQUE_a0, TABLE, s + 3, s + 23)
print("done in", round(time.time() - t1, 2), "s")

In [ ]:
tuples_indices

### Accès par intervalle sur un attribut NON unique
Accès pour rechercher les tuples dont l'attribut indexé n'est **pas** unique et a une valeur comprise dans un intervalle donné.
Votre solution doit prendre en compte les mêmes exigences que dans la question précédente.

In [ ]:
def acces_intervalle_par_index(index, table, borne_inf, borne_sup):
    indice_inf = INDEX_UNIQUE_a0.bisect_left(borne_inf)
    indice_sup = INDEX_UNIQUE_a0.bisect_right(borne_sup)
    tuples = []
    while indice_inf<= indice_sup : 
        cle = index.keys()[indice_inf]
        # devrait être 
        # for tuple in  acces_par_index(index,table,cle) # <=== car c'est un yield
        tuples_indice = acces_par_index(index,table,cle)
        for tuple in tuples_indice : 
            tuples.append(tuple)
        indice_inf +=1
    return tuples




In [ ]:
s = 10 * np.random.randint(nb_valeurs_distinctes / 4)
print("valeur recherchée :", s)

t1 = time.time()
acces_intervalle_par_index(INDEX_a2, TABLE, s + 3, s + 33)
print("done in", round(time.time() - t1, 2), "s")

# Exercice 3 : Mise à jour de données




## Modifier la valeur d'un attribut d'un ou plusieurs tuples

Cela correspond à l'insctruction UPDATE table SET ... WHERE ...



### Modification d'un seul tuple

On donne une valeur *v* de l'attribut clé $a_0$. Ajouter 10 à l'attribut $a_1$. Cela correspond à l'instruction

update T
set a1 = a1+10
where a0 = *v*

Après la modification, accéder aux données pour vérifier que le tuple a bien été modifié. Par exemple, invoquer la fonction
acces_par_index_unique(index, table, v)



In [ ]:
def update_unique(index_unique, table, v)-> bool:
    a0= index_unique[v]
    if a0 is None: 
        return  False
    page,position = a0['page'],a0['position']
    df = pd.read_csv(f'T_pages/page{page}.csv',header=None)
    df.at[position,1] =  int(df.iloc[position][1])+10
    df.to_csv(f'T_pages/page{page}.csv', header=None,index=False) 
    return True

In [ ]:
INDEX_UNIQUE_a0.get(19512870)

In [ ]:
# before
valeur_rechere = 19512870
acces_par_index_unique(INDEX_UNIQUE_a0,TABLE,valeur_rechere)

In [ ]:
update_unique(INDEX_UNIQUE_a0,TABLE, valeur_rechere)
acces_par_index_unique(INDEX_UNIQUE_a0,TABLE,valeur_rechere)


### Modification de plusieurs tuples

On donne une valeur *v* de l'attribut $a_2$ qui n'est pas unique. Ajouter 1 à l'attribut $a_3$ de tous les tuples pour lesquels $a_2 = v$

update T set a3 = a3+1 where a2=v



In [ ]:
def update_plusieurs(index, table, v):
    rows= index[v]
    if rows is None: 
        return  False
    for row in rows : 
        a2 = row
        page,position = a2['page'],a2['position']
        df = pd.read_csv(f'T_pages/page{page}.csv',header=None)
        df.at[position,3] =  int(df.iloc[position][3])+10
        df.to_csv(f'T_pages/page{page}.csv', header=None,index=False) 
    return True


In [ ]:
print(INDEX_a2.get(1235270))

In [ ]:
valeur_rechere_non_unique = 1235270
acces_par_index(INDEX_a2,TABLE,valeur_rechere_non_unique)

In [ ]:
update_plusieurs(INDEX_a2,TABLE,valeur_rechere_non_unique)
acces_par_index(INDEX_a2,TABLE,valeur_rechere_non_unique)

### Modifier l'index en conséquence lorsque l'attribut modifié est indexé
Comerncer par créer un index sur l'attribut $a_3$

L'attribut $a_3$ étant maintenant indexé, la mise à jour de la question précédente implique d'actualiser l'index sur $a_3$ pour que les rowid des tuples qui contenaient l'ancienne valeur de $a_3$ soient associés à la nouvelle valeur de $a_3$.

In [ ]:
INDEX_a3 = creation_index(TABLE,3)

# Exercice 4 : Persistence

Dans cette partie, on veut rendre les index persistents en stockant les entrées triées dans des pages. Cela permet d'utiliser les index plus efficacement en réduisant la durée pour les reconstruire.

## Stockage d'un index unique

Proposez une solution pour stocker les entrées **triées** d'un index dans plusieurs pages avec une taille de page fixée (10 000 rowids par page).
Etudier le cas d'un index unique et celui d'un index non unique

In [ ]:
def index_dir_name(table:str):
    return table+"_index"

In [ ]:
INDEX_a2[150]

In [ ]:
def sauvegarder_index_unique(table:str,index : object,nom_attribut='a0',nb_tuple_par_page=10_000):
  index_name = f'{index_dir_name(table)}/{nom_attribut}'

  os.makedirs(index_name, exist_ok=True)

  # lire le fichier contenant tous les tuples
  attributes = ['value','page','position']
  
  p=0
  lines = []
  for i, key in enumerate(index.keys()):
    # lines.append(line)
    
    page,position = index[key]['page'],index[key]['position']
    lines.append(f'{key},{page},{position}\n')
    if (i+1) % nb_tuple_par_page == 0:

      # créer une page
      p += 1
      with open(f"{index_name}/page{p}.csv", "w") as fp:
        fp.write(','.join(attributes))
        fp.write('\n')
        fp.write(''.join(lines))
      lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(f"{index_name}/page{p}.csv", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p) 
    

In [ ]:
sauvegarder_index_unique(TABLE,INDEX_UNIQUE_a0)

In [ ]:
def sauvegarder_index(table:str,index : object,nom_attribut='a2',nb_tuple_par_page=10_000):
  index_name = f'{index_dir_name(table)}/{nom_attribut}'

  os.makedirs(index_name, exist_ok=True)

  # lire le fichier contenant tous les tuples
  attributes = ['value','page','position']
  
  p=0
  lines = []
  for i, key in enumerate(index.keys()):
    for row in index[key] : 
      page,position = row['page'],row['position']
      lines.append(f'{key},{page},{position}\n')
      if (i+1) % nb_tuple_par_page == 0:
        # créer une page
        p += 1
        with open(f"{index_name}/page{p}.csv", "w") as fp:
          fp.write(','.join(attributes))
          fp.write('\n')
          fp.write(''.join(lines))
        lines = []

  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open(f"{index_name}/page{p}.csv", "w") as fp:
        fp.write(''.join(lines))

  print("nb pages créées :", p) 

In [ ]:
sauvegarder_index(TABLE,INDEX_a2)

Montrez que vous pouvez reconstruire les index à partir des entrées stockées dans des pages.

In [ ]:
def lecture_sequentielle_par_page_index(index_name):
  nb_pages = len(os.listdir(index_name))
  for p in range(1, nb_pages) :
    with open(f"{index_name}/page{p}.csv", "r") as f:
      for i, line in enumerate(f):
        if "value" not in line : 
          value,page,position= line.strip().split(',')
          yield value,page, position

  

In [ ]:

def charger_index_unique(table:str,nom_index='a0'):
    index = {}
    index_name = f'{index_dir_name(table)}/{nom_index}'
    if not os.path.exists(index_name) : 
        return None 
    for value, page, position in lecture_sequentielle_par_page_index(index_name):
        index[int(value)] = {
        'page':int(page),'position' : int(position)
        }
        # print(f"page {page}, ligne {position} :", )
    return sortedcontainers.SortedDict(index)
    
    

In [ ]:
index=charger_index_unique(TABLE)

In [ ]:
index[0]

In [ ]:
INDEX_UNIQUE_a0[0]

In [ ]:

def charger_index(table:str,nom_index='a2'):
    index = {}
    index_name = f'{index_dir_name(table)}/{nom_index}'
    if not os.path.exists(index_name) : 
        return None 
    for value, page, position in lecture_sequentielle_par_page_index(index_name):
        position_page = {'page':int(page),'position' : int(position)}
        if index.get(int(value)) is None : 
            index[int(value)] = [position_page]
        else :
            index[int(value)].append(position_page)
        # print(f"page {page}, ligne {position} :", )
    return sortedcontainers.SortedDict(index)
    
    

In [ ]:
index_a2_chargee = charger_index(TABLE)

In [ ]:
index_a2_chargee[0]

In [ ]:
INDEX_a2[0]

## Adapter en conséquence les opérations de modification de l'index

Illustrer le cas :

update T set a3 = a3+0.5 where a1=v

où la nouvelle valeur $a_3' = a_3 + 0.5$ n'est pas déjà présente dans l'index. Il faut donc insérer une nouvelle clé  dans l'index de l'attribut $a_3$.
On suppose qu'il reste de la place dans une page de l'index pour insérer la nouvelle entrée (on peut avoir jusqu'à 12 000 rowids par page d'index).

In [ ]:
def update_index_unique(index_unique, table, v, new_value):
    # Lecture de l'index existant
    index_name = f'{index_dir_name(table)}/a1'  # Remplacer "a1" par le nom de l'attribut correspondant
    index = charger_index(index_name)

    # Mise à jour de l'index
    if index is None:
        index = sortedcontainers.SortedDict()

    # Supprimer l'ancienne valeur de l'index
    if v in index:
        del index[v]

    # Insérer la nouvelle valeur dans l'index
    page, position = INDEX_UNIQUE_a0[new_value]['page'], INDEX_UNIQUE_a0[new_value]['position']
    index[new_value] = {'page': page, 'position': position}

    # Sauvegarder le nouvel index
    sauvegarder_index_unique(table, index, 'a1')  # Remplacer "a1" par le nom de l'attribut correspondant

# Exercice 5 : Index bitmap
*   Proposer un index ayant une structure matricielle ("bitmap") pour l'attribut $a_5$. Idem pour l'attribut $a_6$.
*   En utilisant les 2 index bitmap, rechercher les tuples de T tels que $a_5 = v_1$ et $a_6 = v_2$ pour deux valeurs $v_1, v_2$ appartenant au domaine de $a_5 \cap a_6$ .



In [ ]:
INDEX_UNIQUE_a0.keys()[10]

In [ ]:
INDEX_UNIQUE_a0.index(100)

In [ ]:
def creation_index_bitmap(table, numero_attribut_i):
   return creation_index(table, numero_attribut_i)

In [ ]:
def rechercher_tuples(table, index_a5, index_a6):
    
    # Récupérer l'intersection des domaines de a5 et a6
    domain_a5 = set(index_a5.keys())
    domain_a6 = set(index_a6.keys())
    intersection_domaines = domain_a5.intersection(domain_a6)
    
    # Parcourir les valeurs possibles dans l'intersection des domaines
    for v1 in intersection_domaines:
        for v2 in intersection_domaines:
            # Récupérer les occurrences de v1 et v2
            occurrences_v1 = index_a5.get(v1, [])
            occurrences_v2 = index_a6.get(v2, [])
            
            # Trouver les pages communes
            pages_communes = {occurrence['page'] for occurrence in occurrences_v1} & {occurrence['page'] for occurrence in occurrences_v2}
            
            # Rechercher les tuples sur les pages communes (même position)
            for page in pages_communes:
                for occurrence_v1 in occurrences_v1:
                    if occurrence_v1['page'] == page:
                        for occurrence_v2 in occurrences_v2:
                            if occurrence_v2['page'] == page and occurrence_v1['position'] == occurrence_v2['position']:
                                yield lecture_tuple(table,page,occurrence_v1['position'])

In [ ]:
INDEX_a5_bitmap = creation_index_bitmap(TABLE,5)

In [ ]:
INDEX_a6_bitmap = creation_index_bitmap(TABLE,6)

In [ ]:
gen = rechercher_tuples(TABLE,INDEX_a5_bitmap,INDEX_a6_bitmap)
   

In [ ]:
next(gen)